Django基础实例_04
====

**URLConf，路由路径，路由与视图，怎样调用视图。**

# 建立请求与视图的对应关系

在一个网站中有若干个视图，用户可能提出不同的请求。

约定某个请求访问适当的视图是网站设计者的重要任务之一。

Django是通过路由路径的设置来完成这个任务的。

在创建Django项目时，在项目包目录下会创建一个很重要的基础文件`setting.py`，这是项目的配置文件，

在该配置文件中，有一个`ROOT_URLCONF`配置项，默认设置为：

`ROOT_URLCONF = 'pyforest.urls'`

它将项目路由的根路径设置为`pyforest.ursl`，也就是说django解析路由地址时，首先访问`pyforest/urls.py`这个路由设置文件，它就是一级路由。

# django的多级路由

所谓的路由，可以简单地理解为网络访问路径。路径上总是有节点、叉路口，在节点、叉路口上，下一步怎么走，就是路由要解决的问题。

django支持多级路由。什么是多级路由呢？

先这样简单地理解：

- 把路由路径的所有定义明确地写在一个路由定义文件里，在每个节点上，都是明确地定义：针对一个请求访问一个具体的视图。这就是一级路由。
- 如果在一级路由的某个节点上，不是明确定义要访问的视图，而是让程序去访问另一个路由文件，这就是多级路由。

为了清晰、有效地组织和管理网站项目文件，提倡将django项目中某个app专用的文件都放在app包目录下。

前面的练习已有一app，为了更好地理解多级路由机制，再创建一个app，命名为`forum`：

`python manage.py startapp forum`

**将`forum`添加到 setting.py的INSTALLED_APPS中。**

在`tnt`, `forum` app包目录下各新建一个路由定义文件：

- `pytforest/tnt/urls.py`
- `pyforest/forum/urls.py`

django程序首先访问的是`ROOT_URLCONF`，也就是`pyforest.urls`，它就是一级路由。

在`pyforest.urls`中再指向`'pyforest/tnt/urls.py'`，或`'pyforest/forum/urls.py'`这就是二级路由。

# 为 'forum' 程序写一个视图

在[Django基础实例-03](Django基础实例-03.ipynb)中已经为`tnt`应用程序创建了一个视图，为`forum`也创建一个视图：

在`forum/views.py`文件中写入如下代码：

```
from django.shortcuts import render
from django.http import HttpRequest, HttpResponse

# Create your views here.

def forumindex(request):
    return HttpResponse('<h1>这里是 Pyhton 森林的论坛！<h1><br><h2>Python 草堂论道!<h2>')

```
为`tnt/views.py`增加两个视图，结果如下：

```
from django.http import HttpRequest, HttpResponse
from django.shortcuts import render

# Create your views here.
def index(request):
    return HttpResponse('<h1>欢迎访问 python 森林！')

def pandasindex(request):
    return HttpResponse('<h2>这里是 pandas 小屋!<h2>')

def djangoindex(request):
    return HttpResponse('<h2>这里是 Django 草堂！')
    
```

# 编写二级路由调用适当的视图

django程序框架的`django.urls`模块有一系列函数专门用于处理路由路径的问题，先了解以下两个，其它的在后面再深入了解：

- path()
- include()

## path()语法

path()函数语法：

`path(route, view, kwargs=None, name=None)`

- `route` 参数是包含URL模式的字符串（本例中是空的字符串 ''）。该字符串可以使用正则表达式、转换器等方法来约定请求的转换规则。先就了解这么多即可，后面再详细讨论它。

- `view` 参数是视图函数。如果这里直接用指向一个视图函数，它就会直接调用该视图。

- `view` 参数也可以是一个django.urls.include（）。当使用include()函数时，将另一个`urls.py`文件作为它的参数，这就是多级路由的概念了。

- `kwargs` 参数允许您将其他参数传递给视图函数或方法。

- `name` 参数，给这个路由定义一个名字，高级篇中再讨论它。

`path()`函数返回的对象作为路由定义文件中`urlpatterns`变量的一个元素。django路由路径解析到这里，会在`urlpatterns`变量中依次查找匹配项，找到匹配项时即停止查找，立即调用找到的path中映射的view。

## include()语法

- `include(module, namespace=None)[source]`
- `include(pattern_list)`
- `include((pattern_list, app_namespace), namespace=None)`

include()是django.urls模块的一个函数，它将Python输入的完整路径带到另一个URLconf模块(urls.py文件)，该模块应该被“包含”在include()函数的括号里面。可选地，还可以指定将包括条目的应用程序命名空间和实例命名空间。

通常，应用程序命名空间应由包含的模块指定（如`tnt.urls`，`tnt`就是应用程序命名空间）。如果设置了应用程序命名空间，则可以使用 namespace 参数设置不同的实例命名空间。

include（）还接受返回URL模式的 iterable（pattern_list，模式列表）作为参数；或包含一个元组作为参数，该元组有两个元素，一个是可迭代的模式列表，第二个是应用程序命名空间。

**inlude()的参数:**

- module – URLconf 模块 (or 模块名)
- namespace (str) – 包含的URL条目的实例命名空间
- pattern_list – path() and/or re_path() 实例组成的可迭代对象（列表）
- app_namespace (str) – 包含的URL条目实例的应用程序命名空间


**在`pytforest/tnt/urls.py`中写入如下代码：**

```
from django.urls import path
from . import views

urlpatterns = [
    
    path('pandas/',views.pandasindex),
    path('django/',views.djangoindex),
        
]
```

**在`pyforest/forum/urls.py`中写入如下代码：**

```
from django.urls import path, include
from . import views

urlpatterns=[

path('',views.forumindex),

]
```

当Django路由访问到这里时，就会加载 `pytforest/tnt/urls.py` 这个 Python 模块，并查找变量`urlpatterns`。

`urlpatterns` 是一个Python列表，`django.urls.path()` 和(或) `django.urls.re_path()`函数的实例组成列表的元素。

# 编写根路由引导访问二级路由

视图和二级路由都已写好，下一步就需要在django一级路由（根路由）中定义路由路径，引导django从一级路由访问二级路由。

django在创建项目时，默认会创建根路由`pyforest/urls.py`，打开该文件，默认已有一些代码：

```
from django.contrib import admin
from django.urls import path

urlpatterns = [
    path('admin/', admin.site.urls),
]
```

django有一个内建的`admin.site.urls`视图，这是django的站点管理界面。对于`admin/`请求，将调用这个视图作为响应。 后面再详细讨论Django的网站管理模块。

对`pyforest/urls.py`文件中的代码做如下修改：

```
from django.urls import path, include
# 为了引用不同目录下的urls，或views，需要先扩展python环境的系统路径
import sys
sys.path.append("..")
import pyforest, tnt, forum
from tnt import views

urlpatterns = [

    path('admin/', admin.site.urls),
    path('index/', tnt.views.index),
    path('tnt/',include('tnt.urls')),
    path('forum',include('forum.urls')),
    
]
```

`include（）`函数允许引用其他`URLconf`。 

**每当Django遇到`include（）`时，它会截断与该点匹配的URL的任何部分，并将剩余的字符串发送到包含的URLconf以供进一步处理。**

include()函数需要另一个路由文件作为参数。注意该参数的写法，它是一个点路径`tnt.urls`，django会自动到各app下查找。所以用`tnt.urls`这样的短路径即可。

**何时使用include（）**

包含其他URL模式时，应始终使用`include（）`。 `admin.site.urls`是唯一的例外。

# 启动服务，浏览index页

`python manage.py runserver`启动django服务。

在浏览器中输入"http://127.0.0.1:8000/index" 

在浏览器就会显示这样的页面：

<img src="../../../pf-img-store/django/example-index.png">

在浏览器中输入" http://127.0.0.1:8000/tnt/pandas "  就会显示这样的页面：

<img src="../../../pf-img-store/django/进阶pandasindex.png">

表明一切正常！梳理一下：

- 定义一个视图函数，它是一个python函数，将请求（实例化前还是一个request形参）与一个响应（内容）对应起来；
- 当用户提出请求时，实参将相应的视图函数实例化；
- 在二级路由中定义一个path()函数用于约定路由路径，将实际的请求与视图函数关联起来，比如'pandas'请求（这是一个二级请求）将调用哪个视图；
- 在根路由中定义一个path()函数，将实际的请求(如'tnt'，这个是一级请求)，通过include()函数指向二级路由。
- django在获得请求时，首先调用django.http模块的HttpRequest类对象对请求进行处理，生成该类的一个实例，并将该实例作为参数传递给views函数的第一个参数，views函数返回一个HttpResponse对象的实例，该实例反馈（呈现）给用户浏览器。
- 至于某个请求调用哪个视图，这就是URLConf的任务，django.urls，先在根路由的urlpatterns变量查找，找到匹配项，如果直接是一个视图，就调用这个视图；如果是一个转向定义，就转到下一级路由，继续匹配查找的步骤。
- 如果是多级请求，在一级路由处理时，会对请求进行截断，把后面传递给二级路由，依次进行下去。

**这就是django的最基本工作机制。**

<img src="../../../pf-img-store/django/基础流程.png">

**现在整个项目的目录结构如下：**

```
I:./pyforest400
│  db.sqlite3
│  manage.py
│
├─forum #这是app包目录
│  │  admin.py
│  │  apps.py
│  │  models.py
│  │  tests.py
│  │  urls.py  #app下的二级路由模块
│  │  views.py
│  │  __init__.py
│  │
├─pyforest #这是项目包目录
│  │  settings.py
│  │  urls.py  #这是默认的项目根路由模块
│  │  wsgi.py
│  │  __init__.py
│
└─tnt #这是另一个app包目录
    │  admin.py
    │  apps.py
    │  models.py
    │  tests.py
    │  urls.py #另一个app下的二级路由模块
    │  views.py
    │  __init__.py
   
```